<a href="https://colab.research.google.com/github/ShivanshPanwar/RevoltronX--Assignment/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas numpy scikit-learn flask nltk


In [3]:
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
# Loading the dataset from colab
recipes = pd.read_csv("/content/RAW_recipes.csv")
interactions = pd.read_csv("/content/RAW_interactions.csv")
recipes = recipes[['id', 'name', 'ingredients']]
interactions = interactions[['user_id', 'recipe_id', 'rating']]

# Handleing the missing values
recipes.dropna(inplace=True)
interactions.dropna(inplace=True)

recipes['ingredients'] = recipes['ingredients'].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", ""))

# Displaying the data
print("Recipes Data Sample:\n", recipes.head())
print("\nUser Interactions Sample:\n", interactions.head())

Recipes Data Sample:
        id                                        name  \
0  137739  arriba   baked winter squash mexican style   
1   31490            a bit different  breakfast pizza   
2  112140                   all in the kitchen  chili   
3   59389                          alouette  potatoes   
4   44061          amish  tomato ketchup  for canning   

                                         ingredients  
0  winter squash, mexican seasoning, mixed spice,...  
1  prepared pizza crust, sausage patty, eggs, mil...  
2  ground beef, yellow onions, diced tomatoes, to...  
3  spreadable cheese with garlic and herbs, new p...  
4  tomato juice, apple cider vinegar, sugar, salt...  

User Interactions Sample:
    user_id  recipe_id  rating
0    38094      40893       4
1  1293707      40893       5
2     8937      44394       4
3   126440      85009       5
4    57222      85009       5


In [11]:
def recommend_recipes(recipe_name, top_n=5):
    indices = recipes_sample[recipes_sample['name'].str.contains(recipe_name, case=False, na=False)].index

    if len(indices) == 0:
        return "Recipe not found."
    idx = indices[0]
    tfidf_matrix_index = recipes_sample.index.get_loc(idx)

    distances, indices = nn_model.kneighbors(tfidf_matrix[tfidf_matrix_index], n_neighbors=top_n+1)

    return recipes_sample.iloc[indices[0][1:]][['name', 'ingredients']]

print("\nRecommended Recipes:\n", recommend_recipes("chocolate cake"))


Recommended Recipes:
                                              name  \
65676                    crock pot chocolate cake   
89278                fudgy chocolate chip muffins   
73048          double chocolate sour cream scones   
1091    2 minute eggless microwave chocolate cake   
216463                  triple chocolate biscotti   

                                              ingredients  
65676   brown sugar, flour, cocoa powder, unsweetened ...  
89278   unsweetened applesauce, quick-cooking oats, ma...  
73048   all-purpose flour, whole wheat flour, brown su...  
1091    all-purpose flour, unsweetened cocoa powder, v...  
216463  all-purpose flour, unsweetened cocoa powder, b...  


In [ ]:
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

recipes = pd.read_csv("/content/RAW_recipes.csv")
interactions = pd.read_csv("/content/RAW_interactions.csv")
recipes = recipes[['id', 'name', 'ingredients']]
interactions = interactions[['user_id', 'recipe_id', 'rating']]

recipes.dropna(inplace=True)
interactions.dropna(inplace=True)

recipes['ingredients'] = recipes['ingredients'].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", ""))

print("Recipes Data Sample:\n", recipes.head())
print("\nUser Interactions Sample:\n", interactions.head())

rating_matrix = interactions.pivot(index='user_id', columns='recipe_id', values='rating').fillna(0)

sparse_matrix = csr_matrix(rating_matrix.values)

svd = TruncatedSVD(n_components=50, random_state=42)
user_factors = svd.fit_transform(sparse_matrix)
recipe_factors = svd.components_

def predict_rating(user_id, recipe_id):
    if user_id not in rating_matrix.index or recipe_id not in rating_matrix.columns:
        return "User or recipe not found."

    user_idx = rating_matrix.index.get_loc(user_id)
    recipe_idx = rating_matrix.columns.get_loc(recipe_id)

    return np.dot(user_factors[user_idx], recipe_factors[:, recipe_idx])

print("\nPredicted Rating for User 1 & Recipe 100:\n", predict_rating(1, 100))

<ipython-input-2-4d18b52d0e39>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes.dropna(inplace=True)


Recipes Data Sample:
        id                                        name  \
0  137739  arriba   baked winter squash mexican style   
1   31490            a bit different  breakfast pizza   
2  112140                   all in the kitchen  chili   
3   59389                          alouette  potatoes   
4   44061          amish  tomato ketchup  for canning   

                                         ingredients  
0  winter squash, mexican seasoning, mixed spice,...  
1  prepared pizza crust, sausage patty, eggs, mil...  
2  ground beef, yellow onions, diced tomatoes, to...  
3  spreadable cheese with garlic and herbs, new p...  
4  tomato juice, apple cider vinegar, sugar, salt...  

User Interactions Sample:
    user_id  recipe_id  rating
0    38094      40893       4
1  1293707      40893       5
2     8937      44394       4
3   126440      85009       5
4    57222      85009       5


<ipython-input-2-4d18b52d0e39>:30: PerformanceWarning: The following operation may generate 52481995090 cells in the resulting pandas object.
  rating_matrix = interactions.pivot(index='user_id', columns='recipe_id', values='rating').fillna(0)


In [3]:
def hybrid_recommend(user_id, recipe_name, top_n=5):
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.neighbors import NearestNeighbors

    global recipes
    recipes_sample = recipes.copy()

    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(recipes_sample['ingredients'])

    nn_model = NearestNeighbors(metric='cosine', algorithm='brute')
    nn_model.fit(tfidf_matrix)

    def recommend_recipes(recipe_name, top_n=5):

        indices = recipes_sample[recipes_sample['name'].str.contains(recipe_name, case=False, na=False)].index

        if len(indices) == 0:
            return "Recipe not found."

        idx = indices[0]

        tfidf_matrix_index = recipes_sample.index.get_loc(idx)

        distances, indices = nn_model.kneighbors